# 0. Short description 

In the first part of the lab you are going to implement a standard document processing pipeline and then build a simple search engine based on it:
- starting from crawling documents, 
- then building an inverted index,
- and answering queries using this index.

Second part is devoted to spellchecking. You will implement some functions related to spellchecking.
- K-gram index
- weighted editorial distance
- Norvig spellchecker


# 1. [60] Building inverted index and answering queries

## 1.1. [10] Preprocessing

First, we need a unified approach to documents preprocessing. Implement a class responsible for that. Complete the code for given functions (most of them are just one-liners) and make sure you pass the tests. Make use of `nltk` library or any other you know.

In [1]:
#!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize 

class Preprocessor:
    
    def __init__(self):
        self.stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                      'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with'}
        self.ps = nltk.stem.PorterStemmer()

    
    def tokenize(self, text):
        #TODO word tokenize text using nltk lib
        return word_tokenize(text) 

    
    def stem(self, word, stemmer):
        #TODO stem word using provided stemmer
        return stemmer.stem(word)

    
    def is_apt_word(self, word):
        #TODO check if word is appropriate - not a stop word and isalpha, 
        # i.e consists of letters, not punctuation, numbers, dates
        return True if word not in self.stop_words and word.isalpha() else False
    
    def preprocess(self, text):
        #TODO combine all previous methods together: tokenize lowercased text 
        # and stem it, ignoring not appropriate words
        tokenized_text = self.tokenize(text)
        return [self.stem(word.lower(),self.ps) for word in tokenized_text if self.is_apt_word(word.lower())]

[nltk_data] Downloading package punkt to /home/olya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 1.1.1. Tests

In [2]:
prep = Preprocessor()
text = 'To be, or not to be, that is the question'

assert prep.tokenize(text) == ['To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question']
assert prep.stem('retrieval', prep.ps) == 'retriev'
assert prep.is_apt_word('qwerty123') is False
assert prep.preprocess(text) == ['or', 'not', 'question']

## 1.2. [35] Crawling and Indexing

### 1.2.1. [10] Base classes

Here are some base classes you will need for writing an indexer. The code from the first lab's solution is given, still you need to change some of it, namely, the `parse` function (it is also possible to use your own implementation from the first homework, but make sure that it is correct). The reason is it always makes complete parsing, which we want to avoid when we only need links, for example, or a specific portion of text.

In [201]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import os


class Document:

    def __init__(self, url):
        self.url = url

    def download(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                self.content = response.content
                return True
            else:
                return False
        except:
            return False

    def persist(self, path):
        with open(os.path.join(path, quote(self.url).replace('/', '_')), 'wb') as f:
            f.write(self.content)


class HtmlDocument(Document):

    def normalize(self, href):
        if href is not None and href[:4] != 'http':
            href = urllib.parse.urljoin(self.url, href)
        return href

    def parse(self, imgs = False, links = True, text=False):
        #TODO change this method
        
        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True
            
        
        model = BeautifulSoup(self.content, 'html.parser')
        
        self.anchors = []
        self.images = []
        self.text = []
        
        if links:
            a = model.find_all('a')
            for anchor in a:
                href = self.normalize(anchor.get('href'))
                text = anchor.text
                self.anchors.append((text, href))
                
            i = model.findAll('iframe')
            for iframe in i:
                url = self.normalize(iframe.get('src'))
                text = iframe.getText()
                self.anchors.append((text, url))
        
        if imgs:
            i = model.find_all('img')
            for img in i:
                href = self.normalize(img.get('src'))
                self.images.append(href)
        
        if text:
            texts = model.findAll(text=True)
            visible_texts = filter(tag_visible, texts)  
            self.text = u" ".join(t.strip() for t in visible_texts)

### 1.2.2. [20] Main class

The main indexer logic is here. We organize it as a crawler generator that adds certain visited pages to inverted index and saves them on disk. 

- `crawl_generator_for_index` method crawles the given website doing BFS, starting from `source` within given `depth`. Considers only inner pages (of a form https://www.reuters.com/...) for visiting. To speed up, doesn't consider for visiting pages with content type other than html: '.pdf', '.mp3', '.avi', '.mp4', '.txt'. If encounters an article page (of a form https://www.reuters.com/article/...), saves its content in a file in `collection_path` folder and populates the inverted index calling `index_doc` method. When done, saves on disk three resulting dictionaries:
    - `doc_urls`, `doc_id:url`
    - `index`, `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    - `doc_lengths`, `doc_id:doc_length` 

    `limit` parameter is given for testing - if not `None`, break the loop when number of saved articles exceeds the `limit` and return without writing dictionaries to disk.
    
    
- `index_doc` method parses and preprocesses the content of a `doc` and adds it to the inverted index. Also keeps track of document lengths in a `doc_lengths` dictionary.

Your crawler have to print visited urls as it runs.

In [250]:
from collections import Counter
from queue import Queue
import pickle
import os
import requests

class Indexer:

    def __init__(self):      
        # dictionaries to populate
        self.doc_urls = {}        
        self.index = {}
        self.doc_lengths = {}
        # preprocessor
        self.prep = Preprocessor()
        self.queue = Queue()
        self.queued_urls = set()
        self.visited_urls = set()
        
        
    
    def crawl_generator_for_index(self, source, depth, collection_path="collection", limit=None):        
        #TODO generate url-s for visiting
        self.limit = limit
        self.create_queue(source,depth)
        
        if not os.path.exists(collection_path):
                os.mkdir(collection_path)
                
        self.create_queue(source,depth)
        print(self.queue.qsize())
        #print(self.visited_urls)
        
        self.saved_articles = 0
        while not self.queue.empty():
            if limit is not None and self.saved_articles >= limit:
                break
            url = self.queue.get()
            html_doc = HtmlDocument(url)
            if html_doc.download():
                no_error_flag = 0
                try:
                    html_doc.persist(collection_path)
                    no_error_flag=1
                except:
                    print("could not persist ", url)
                if no_error_flag==1:
                    self.saved_articles+=1
                    doc_id = len(self.doc_urls)+1
                    self.doc_urls[doc_id]=url
        for d_id in self.doc_urls.keys():
            u = self.doc_urls[d_id]
            doc = HtmlDocument(u)
            doc.download()
            self.index_doc(doc, d_id)
            yield Document(u)
        
        # save results for later use
        with open('doc_urls.p', 'wb') as fp:
            pickle.dump(self.doc_urls, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open('inverted_index.p', 'wb') as fp:
            pickle.dump(self.index, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open('doc_lengths.p', 'wb') as fp:
            pickle.dump(self.doc_lengths, fp, protocol=pickle.HIGHEST_PROTOCOL)
            
    def valid_url(self,url):
        if url is None or '.mp3' in url or '.pdf' in url or '.avi' in url or '.mp4' in url or '.txt' in url:
            return False
        if 'https://www.reuters.com/' not in url:
            return False
        try:
            resp = requests.get(url,allow_redirects=True,timeout=3)
            if resp is not None and resp.status_code>=200 and resp.status_code < 300:
                return True
        except:
            return False
        return False
        
    def create_queue(self,source,depth):
        print(source)
        if depth==1:
            if source not in self.queued_urls and self.valid_url(source) and 'https://www.reuters.com/article/' in source:
                self.queued_urls.add(source)
                self.queue.put(source)
        else:
            if source not in self.queued_urls and self.valid_url(source):
                doc = HtmlDocument(source)
                doc.download()
                doc.parse(imgs=False,links=True,text=False)
                if 'https://www.reuters.com/article/' in source:
                    self.queue.put(source)
                    print('Added to queue ', source)
                for anchor in doc.anchors:
                    _, url = anchor
                    if url not in self.visited_urls:
                        self.visited_urls.add(url)
                        self.create_queue(url,depth-1)
            
        
    def index_doc(self, doc, doc_id):
        #TODO add documents to index 
        doc.parse(imgs=False,links=False,text=True)
        terms = self.prep.preprocess(doc.text)
        self.doc_lengths[doc_id]=len(terms)
        for term in terms:
            if term not in self.index.keys():
                if self.limit is not None:
                    self.index[term] = [0]+[[d_id, 0] for d_id in range(1,(min(len(self.doc_urls),self.limit)+1))]
                else:
                    self.index[term] = [0]+[[d_id, 0] for d_id in range(1,len(self.doc_urls)+1)]
            self.index[term][0]+=1
            self.index[term][doc_id] = list(self.index[term][doc_id])
            self.index[term][doc_id][1]+=1
            self.index[term][doc_id] = tuple(self.index[term][doc_id])

### 1.2.3. Tests

Please make sure your crawler prints out urls with `print(k, c.url)`

In [247]:
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/technology", 2, "test_collection", 5):
    print(k, c.url)
    k += 1

assert type(indexer.index) is dict
assert type(indexer.index['reuter']) is list
assert type(indexer.index['reuter'][0]) is int
assert type(indexer.index['reuter'][1]) is tuple

11
https://www.reuters.com/article/us-usa-semiconductors/intel-in-talks-to-produce-chips-for-automakers-within-six-to-nine-months-ceo-idUSKBN2BZ2C4
https://www.reuters.com/article/us-gamestop-ceo-exclusive/exclusive-gamestop-initiates-search-for-new-ceo-sources-idUSKBN2BZ290
https://www.reuters.com/article/us-usa-biden-chips/white-house-zeros-in-on-chip-shortage-in-meeting-with-company-officials-idUSKBN2BZ118
https://www.reuters.com/article/us-nvidia-forecast/nvidia-expects-first-quarter-sales-to-exceed-5-3-billion-idUSKBN2BZ2CB
https://www.reuters.com/article/us-nuance-commns-m-a-microsoft/microsoft-to-buy-ai-firm-nuance-for-16-billion-to-boost-healthcare-business-idUSKBN2BZ1FS
1 https://www.reuters.com/article/us-usa-semiconductors/intel-in-talks-to-produce-chips-for-automakers-within-six-to-nine-months-ceo-idUSKBN2BZ2C4
2 https://www.reuters.com/article/us-gamestop-ceo-exclusive/exclusive-gamestop-initiates-search-for-new-ceo-sources-idUSKBN2BZ290
3 https://www.reuters.com/article/u

### 1.2.4. Building an index

In [248]:
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/", 3, "docs_collection"):
    print(k, c.url)
    k += 1

56
https://www.reuters.com/article/us-usa-minnesota-shooting/police-say-officer-mistakenly-fired-gun-instead-of-taser-in-killing-of-wright-idUSKBN2BZ21A
https://www.reuters.com/article/us-usa-race-georgefloyd/a-big-mommas-boy-george-floyds-brother-recalls-childhood-at-chauvin-murder-trial-idUSKBN2BZ21M
https://www.reuters.com/article/us-usa-police-virginia/virginia-police-officer-accused-of-assaulting-u-s-army-officer-fired-idUSKBN2BZ0HM
https://www.reuters.com/article/us-shooting-knoxville/reports-of-shooting-at-knoxville-tennessee-high-school-with-multiple-victims-police-idUSKBN2BZ2L4
https://www.reuters.com/article/us-usa-guns/exclusive-biden-plans-nominations-to-two-top-politically-sensitive-legal-posts-idUSKBN2BU2QP
https://www.reuters.com/article/us-iran-nuclear-natanz/iran-blames-israel-for-natanz-nuclear-plant-outage-vows-revenge-idUSKBN2BZ0HD
https://www.reuters.com/article/us-gamestop-ceo-exclusive/exclusive-gamestop-initiates-search-for-new-ceo-sources-idUSKBN2BZ290
https://

4 https://www.reuters.com/article/us-shooting-knoxville/reports-of-shooting-at-knoxville-tennessee-high-school-with-multiple-victims-police-idUSKBN2BZ2L4
5 https://www.reuters.com/article/us-usa-guns/exclusive-biden-plans-nominations-to-two-top-politically-sensitive-legal-posts-idUSKBN2BU2QP
6 https://www.reuters.com/article/us-iran-nuclear-natanz/iran-blames-israel-for-natanz-nuclear-plant-outage-vows-revenge-idUSKBN2BZ0HD
7 https://www.reuters.com/article/us-gamestop-ceo-exclusive/exclusive-gamestop-initiates-search-for-new-ceo-sources-idUSKBN2BZ290
8 https://www.reuters.com/article/us-tech-antitrust-hawley/u-s-senator-wants-to-ban-big-tech-from-buying-anything-ever-again-idUSKBN2BZ2MD?il=0
9 https://www.reuters.com/article/us-usa-stocks/wall-street-ends-lower-as-investors-await-earnings-inflation-data-idUSKBN2BZ16Y?il=0
10 https://www.reuters.com/article/us-credit-suisse-gp-bonuses/credit-suisse-cuts-bonuses-following-archegos-loss-ft-idUSKBN2BZ2L8?il=0
11 https://www.reuters.com/ar

### 1.2.5. [5] Index statistics

Load an index and print the statistics.

In [249]:
# load index, doc_lengths and doc_urls
with open('inverted_index.p', 'rb') as fp:
    index = pickle.load(fp)
with open('doc_lengths.p', 'rb') as fp:
    doc_lengths = pickle.load(fp)
with open('doc_urls.p', 'rb') as fp:
    doc_urls = pickle.load(fp)
    
    
print('Total index length', len(index))
print('\nTop terms by number of documents they apperared in:')
sorted_by_n_docs = sorted(index.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
print([(sorted_by_n_docs[i][0], len(sorted_by_n_docs[i][1])) for i in range(20)])
print('\nTop terms by overall frequency:')
sorted_by_freq = sorted(index.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True)
print([(sorted_by_freq[i][0], sorted_by_freq[i][1][0]) for i in range(20)])

Total index length 3042

Top terms by number of documents they apperared in:
[('zurich', 57), ('zionist', 57), ('zieminski', 57), ('zhu', 57), ('zhongguancun', 57), ('zhejiang', 57), ('zero', 57), ('zarif', 57), ('zammit', 57), ('zainab', 57), ('yuan', 57), ('yourself', 57), ('your', 57), ('younger', 57), ('young', 57), ('you', 57), ('york', 57), ('yingzhi', 57), ('yield', 57), ('yet', 57)]

Top terms by overall frequency:
[('s', 312), ('reuter', 264), ('said', 263), ('not', 155), ('all', 133), ('report', 119), ('delay', 115), ('advertis', 113), ('thomson', 112), ('use', 98), ('have', 92), ('new', 90), ('monday', 87), ('compani', 87), ('after', 86), ('billion', 80), ('our', 79), ('support', 77), ('we', 76), ('see', 76)]


## 1.3. [15] Answering a query (finally)

Now, given that we already have built the inverted index, it's time to utilize it for answering user queries. In this class there are two methods you need to implement:
- `boolean_retrieval`, the simplest form of document retrieval which returns a set of documents such that each one contains all query terms. Returns a set of document ids. Refer to *ch.1* of the book for details;
- `okapi_scoring`, Okapi BM25 ranking function - assigns scores to documents in the collection that are relevant to the user query. Returns a dictionary of scores, `doc_id:score`. Read about it in [Wikipedia](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) and implement accordingly.

Both methods accept `query` parameter in a form of a dictionary, `term:frequency`

In [6]:
from collections import Counter
import math

class QueryProcessing:
    
    @staticmethod
    def prepare_query(raw_query):
        prep = Preprocessor()
        # pre-process query the same way as documents
        query = prep.preprocess(raw_query)
        # count frequency
        return Counter(query)
    
    @staticmethod
    def boolean_retrieval(query, index):
        #TODO retrieve a set of documents containing all query terms
        results = set([index[list(query)[0]][doc_id][0] for doc_id in range(1,len(index[list(query)[0]])) if index[list(query)[0]][doc_id][1]>0])
        for key in list(query)[1:]:
            results = results&set([index[key][doc_id][0] for doc_id in range(1,len(index[key])) if index[key][doc_id][1]>0])
        return results

    
    @staticmethod
    def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):
        #TODO retrieve relevant documents with scores
        docs_num = len(doc_lengths)
        avgdl = sum(doc_lengths.values())/docs_num
        results = {}
        for term in list(query):
            for pair in index[term][1:]:
                doc_id = pair[0]
                if doc_id not in results.keys():
                    results[doc_id]=0
                docs_num_with_term = len(index[term])-1
                idf = math.log((docs_num-docs_num_with_term+0.5)/(docs_num_with_term+0.5)+1)
                calc = (pair[1]*(k1+1))/(pair[1]+k1*(1-b+(b*doc_lengths[doc_id]/avgdl)))
                results[doc_id]+=idf*calc
        return results
                

### 1.3.1. Tests

In [7]:
test_doc_lengths = {1: 20, 2: 15, 3: 10, 4:20, 5:30}
test_index = {'x': [2, (1, 1), (2, 1)], 'y': [2, (1, 1), (3, 1)], 'z': [3, (2, 1), (4,2)]}


test_query1 = QueryProcessing.prepare_query('x z')
test_query2 = QueryProcessing.prepare_query('x y')


assert QueryProcessing.boolean_retrieval(test_query1, test_index) == {2}
assert QueryProcessing.boolean_retrieval(test_query2, test_index) == {1}
okapi_res = QueryProcessing.okapi_scoring(test_query2, test_doc_lengths, test_index)
assert all(k in okapi_res for k in (1, 2, 3))
assert not any(k in okapi_res for k in (4, 5))
assert okapi_res[1] > okapi_res[3] > okapi_res[2]

# 2. [40] Sorri not veri gud in inglish

Have you ever googled someone's name without knowing exactly how it should be written? Were you ever reluctant to look up the correct spelling of a query you typed? Or just unable to type properly because of being in a rush? Modern search engines usually do a pretty good job in deciphering defective user input. In order to be able to do that, a good spell-checking mechanism should be incorporated into a search procedure. Today we will take one step further towards building a good search engine and work on tolerant retrieval with respect to user queries. We will consider two cases:

1. Users know that they don't know the correct spelling OR they want to get the results that follow some known pattern, so the use so called wildcards - queries like `retr*val`;
2. Users don't know the correct spelling OR they don't care OR they are in a rush OR they expect that mistakes will be corrected OR /your option/... so they make mistakes and we need to handle them using:

    2.1. Trigrams with Jaccard coefficient;
    
    2.2. Simple spellchecker by Peter Norvig with QWERTY weights;

## 2.1. [25] Handling wildcards

We will handle wildcard queries using k-grams. K-gram is a list of consecutive k chars in a string - i.e., for the word *'star'*, it will be '*\$st*', '*sta*', '*tar*', and '*ar$*', if we take k=3. Take a look at [book](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf) *chapter 3.2.2* to understand how k-grams can help efficiently match a wildcard against dictionary words. Here we will only consider wildcards with star symbols (may be multiple).

Notice that for building k-grams index, **we will need a vocabulary of original word forms** to compare words in user input to the vocabulary of "correct" words (think why inverted index which we built for stemmed words doesn't work here).   

You need to implement the following:

- `build_inverted_index_orig_forms` - creates inverted index of original world forms from `facts` list, which is already given to you.  
    Output format: `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    

- `build_k_gram_index` - creates k-gram index which maps every k-gram encountered in facts collection to a list of words containing this k-gram. Use the abovementioned inverted index of original words to construct this index.  
    Output format: `'k_gram': ['word1_with_k_gram', 'word2_with_k_gram', ...]`
    
    
- `generate_wildcard_options` - produce a list of vocabulary words matching given wildcard by intersecting postings of k-grams present in the wildcard (refer to *ch 3.2.2*). 

- `search_wildcard` - return list of facts that contain the words matching a wildcard query.


We will use the dataset with curious facts for testing.

### 2.1.1. Downloading the dataset

In [8]:
import urllib.request
data_url = "https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt"
local_filename, headers = urllib.request.urlretrieve(data_url)

facts = []
with open(local_filename) as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n'))
        
print(*facts[-5:], sep='\n')

151. Women have twice as many pain receptors on their body than men. But a much higher pain tolerance.
152. There are more stars in space than there are grains of sand on every beach in the world.
153. For every human on Earth there are 1.6 million ants.
154. The total weight of all those ants, however, is about the same as all the humans.
155. On Jupiter and Saturn it rains diamonds.


### 2.1.2. [25] Implementation of search

In [9]:
import re
from nltk import ngrams

def build_inverted_index_orig_forms(documents):
    #TODO build an inverted index of original word forms 
    # (without stemming, just word tokenized and lowercased) 
    prep = Preprocessor()
    inverted_index = {}
    for doc_id in range(len(documents)):
        terms = [word.lower() for word in prep.tokenize(documents[doc_id]) if prep.is_apt_word(word)]
        #print(terms)
        for term in terms:
            if term not in inverted_index.keys():
                inverted_index[term] = [0]+[[d_id, 0] for d_id in range(1,len(documents)+1)]
            inverted_index[term][0]+=1
            #print(inverted_index[term], doc_id+1)
            inverted_index[term][doc_id+1] = list(inverted_index[term][doc_id+1])
            inverted_index[term][doc_id+1][1]+=1
            inverted_index[term][doc_id+1] = tuple(inverted_index[term][doc_id+1])
    return inverted_index


def build_k_gram_index(inverted_index, k):
    #TODO build index of k-grams for dictionary words. 
    # Padd with '$' ($word$) before splitting to k-grams    
    k_gram_index = {}
    for word in list(inverted_index):
        word_for_kgram = '$'+word+'$'
        k_grams = ngrams(word_for_kgram,k)
        for kgram in k_grams:
            kgram = "".join(kgram)
            if kgram not in k_gram_index.keys():
                k_gram_index[kgram] = []
            if word not in k_gram_index[kgram]:
                k_gram_index[kgram].append(word)
    return k_gram_index


def generate_wildcard_options(wildcard, k_gram_index, inverted_index):
    #TODO for a given wildcard return all words matching it using k-grams
    # refer to book chapter 3.2.2
    # don't forget to pad wildcard with '$', when appropriate    
    parts = str('$'+wildcard+'$').split('*')
    k = len(list(k_gram_index)[0])
    results = set()
    for part in parts:
        k_grams_for_search = set()
        l=0
        r=l+k
        while l in range(0,len(part)+1) and r in range(0,len(part)+1):
            k_grams_for_search.add(part[l:r])
            l+=1
            r=l+k
        for kgram in k_grams_for_search:
            if len(results)==0:
                results = set(k_gram_index[kgram])
            else:
                results = results&set(k_gram_index[kgram])
    return [result for result in results if re.match(wildcard.replace('*','.*'), result)]


def search_wildcard(wildcard, k_gram_index, index, docs):
    #TODO retrive list of documnets (facts) that contain words matching wildcard
    suggestions = generate_wildcard_options(wildcard, k_gram_index, index)
    results = []
    for word in suggestions:
        results = results+ [docs[pair[0]-1] for pair in index[word][1:] if pair[1] > 0]
    return results

### 2.1.3. Tests

In [10]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)

wildcard = "re*ed"

wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index_orig_forms)
print(wildcard_options)
assert(len(wildcard_options) >= 3)
assert("red" not in wildcard_options) 

wildcard_results = search_wildcard(wildcard, k_gram_index, index_orig_forms, facts)
# some pretty printing
for r in wildcard_results:
    # highlight terms for visual evaluation
    for term in wildcard_options:
        r = re.sub(r'(' + term + ')', r'\033[1m\033[91m\1\033[0m', r, flags=re.I)
    print(r)

assert(len(wildcard_results) >=3)

assert "13. James Buchanan, the 15th U.S. president continuously bought slaves with his own money in order to free them." in search_wildcard("pres*dent", k_gram_index, index_orig_forms, facts)
assert "40. 9 out of 10 Americans are deficient in Potassium." in search_wildcard("p*tas*um", k_gram_index, index_orig_forms, facts)
assert "61. A man from Britain changed his name to Tim Pppppppppprice to make it harder for telemarketers to pronounce." in search_wildcard("*price", k_gram_index, index_orig_forms, facts)

['recorded', 'received', 'reduced']
4. The largest recorded snowflake was in Keogh, MT during year 1887, and was 15 inches wide.
102. More than 50% of the people in the world have never made or received a telephone call.
134. A person can live without food for about a month, but only about a week without water. If the amount of water in your body is reduced by just 1%, you'll feel thirsty. If it's reduced by 10%, you'll die.


## 2.2. [15] Handling typos

### 2.2.0. Dataset 

Download github typo dataset from [here](https://github.com/mhagiwara/github-typo-corpus).
Load it with this code:

In [11]:
!pip install jsonlines

Defaulting to user installation because normal site-packages is not writeable


In [12]:
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] != 'eng':
                other_langs.add(edit['src']['lang'])
                continue

            if edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((edit['src']['text'], edit['tgt']['text']))
                
print(f"Dataset size = {len(dataset)}")

Dataset size = 245909


#### Explore sample typos
Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [13]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

        """Make am instance. =>         """Make an instance.
* travis: test agains Node.js 11 => * travis: test against Node.js 11
The parser receive a string and returns an array inside a user-provided  => The parser receives a string and returns an array inside a user-provided 
CSV data is send through the `write` function and the resulted data is obtained => CSV data is sent through the `write` function and the resulting data is obtained
One useful function part of the Stream API is `pipe` to interact between  => One useful function of the Stream API is `pipe` to interact between 
source to a `stream.Writable` object destination. This example available as  => source to a `stream.Writable` object destination. This example is available as 
`node samples/pipe.js` read the file, parse its content and transform it. => `node samples/pipe.js` and reads the file, parses its content and transforms it.
Most of the generator is imported from its parent project [CSV][csv] in a effort  => Most o

#### 2.2.0.1. Build a dataset vocabulary
Here we prepare a vocabulary for spellchecker testing and for estimating overall correction quality. Consider only word-level. Be carefull, there is markdown (e.g. \`name\`. \[url\]\(http://url)) and comment symbols (\#, //, \*).

In [14]:
def sent_to_words(sent):
    # splits sentence to words, filtering out non-alphabetical terms
    words = nltk.word_tokenize(sent)    
    words_filtered = filter(lambda x: x.isalpha(), words)
    return words_filtered

In [15]:
vocabulary = Counter()
for pair in dataset:
    for word in sent_to_words(pair[1].lower()):
        vocabulary[word] += 1
len(vocabulary)

63724

In [16]:
from itertools import islice
print(list(islice(vocabulary.items(), 10)))

[('function', 6193), ('de', 82), ('deutsch', 4), ('nocomments', 2), ('you', 42075), ('can', 26027), ('disable', 532), ('comments', 360), ('for', 44756), ('the', 207017)]


### 2.2.1. [15] Implement context-independent spellcheker (Trigrams with Jaccard coefficient) ##

In [190]:
def fix_typo_kgram(word, k_gram_index) -> list:
    #TODO return best matches with respect to Jaccard index   
    k = len(list(k_gram_index)[0])
    word_for_kgram = '$'+word+'$'
    word_kgrams = ngrams(word_for_kgram,k)
    
    results = []
    suggestions = []
    for kgram in word_kgrams:
        if ("".join(kgram)) in k_gram_index.keys():
            suggestions = suggestions + k_gram_index["".join(kgram)]
    suggestions = Counter(suggestions)
    for suggestion in list(suggestions):
        results.append((suggestion, jaccard_coefficient(word, suggestion, suggestions, k)))
    results.sort(key = lambda x: x[1], reverse = True)
    return [result[0] for result in results[:3]]

def jaccard_coefficient(word, suggestion, suggestions, k):
    jc = suggestions[suggestion]/((len(word)+2-k+1)+(len(suggestion)+2-k+1)-suggestions[suggestion])
    return jc
    

### 2.2.2. Tests

In [56]:
# tests

k_gram_index_github = build_k_gram_index(vocabulary, 3)
print(fix_typo_kgram("enouh", k_gram_index_github)[:20])
assert "enough" in fix_typo_kgram("enouh", k_gram_index_github), "Assert k-gram failed"

['enough', 'eno', 'enought']


## 2.3. [Bonuses]

### 2.3.1. [Bonus] QWERTY - Editorial distance

Write code to compute weighted QWERTY-editorial distance.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/KB_United_States.svg/640px-KB_United_States.svg.png" width="640"/> 

Use this image to prepare weight function:
- all letter pairs which share vertical border will get 0.5 multiplier **for replace** (`df`, `cv`, `ui`, ...)
- all letter pairs which share at least some horizontal border will get 0.7 multiplier **for replace** (`dc`, `dr`, `km`, ...)
- other operations are not scaled (x1 multiplier).

In [127]:
def generate_neighbors(keyboard):
    result = []
    for l in range(len(keyboard)-1):
        result.append(keyboard[l]+keyboard[l+1])
        result.append(keyboard[l+1]+keyboard[l])
    return result
        
vertical_neighbors = set()
horizontal_neighbors = set()

for s in ['qwertyuiop[]\\', "asdfghjkl;'",'zxcvbnm,./','1234567890-=']:
    horizontal_neighbors = horizontal_neighbors.union(set(generate_neighbors(s)))
          
for s in ['1qaz','2q','3w','2w','3e','4e','4r','5r','5t','6t','6y',
          '7y','7u','8u','8i','9i','9o','0o','0p','-p','-[','=[','=]',
         '2wsx','3edc','4rfv','5tgb','6yhn','7ujm','8ik,','9ol.','0p;/',"-['']",
         'wa','es','rd','tf','yg','uh','ij','ok','pl','[;',"]'",
         'sz','dx','fc','gv','hb','jn','km','l,',';.',"'/"]:
    vertical_neighbors = vertical_neighbors.union(set(generate_neighbors(s)))

def replace_weight(let1, let2) -> float:
    # TODO what is the weight for a pair of letters being replaced? Note this function should be commutative
    if str(let1+let2) in horizontal_neighbors:
          return 0.5
    if str(let1+let2) in vertical_neighbors:
        return 0.7
    return 1

def qwerty_edit_dist(s1, s2) -> float:
    # TODO compute the Damerau-Levenshtein distance between two given strings (s1 and s2)
          
    #SOURCE: https://www.guyrutenberg.com/2008/12/15/damerau-levenshtein-distance-in-python/ 
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1,lenstr2+1):
        d[(-1,j)] = j+1

    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = replace_weight(s1[i],s2[j])
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + cost) # transposition

    return d[lenstr1-1,lenstr2-1]

In [128]:
# tests

assert qwerty_edit_dist("korrectud", "corrected") == 2.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("soem", "some") == 1.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("one", "one") == 0.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fihure", "figure") == 0.5, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fivure", "figure") == 0.7, "Edit distance is computed incorrectly"

### 2.3.2. [Bonus] Norvig's spellchecker with QWERTY weights

You can base your code on [Norvig's corrector](https://norvig.com/spell-correct.html), but you should be sure you account the fact, that typos for close letters cost less. This should be considered in ranking.

In [185]:
#SOURCE: https://norvig.com/spell-correct.html
import requests
def words(text): return re.findall(r'\w+', text.lower())
WORDS = Counter(words(open('big.txt').read()))

def P(word, candidate, N=sum(WORDS.values())): 
    "Probability of `word`."
    p = WORDS[candidate]
    #print(word, candidate, p, qwerty_edit_dist(word, candidate))
    return bool(p)*(1/(1+qwerty_edit_dist(word, candidate)))

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) | known(edits1(word)) | known(edits2(word)) | set([word]))

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def fix_typo_qwerty_norvig(word) -> str:
    #TODO return best matching result for the word
    results = [(candidate,P(word,candidate)) for candidate in candidates(word)]
    results = sorted(results, key = lambda x: x[1], reverse = True)
#     print(known(['confidence']))
#     print(candidates(word))
#     print(results)
    return results[0][0]


In [186]:
# tests

assert fix_typo_qwerty_norvig("korrectud") == "corrected", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("speling") == "spelling", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("condidence") == "confidence", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fpx") == "fox", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fux") == "fix", "Norvig's correcter doesn't work"

### 2.3.3. Estimate quality of functions

In [192]:
norvig, kgram = 0, 0
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    words = sent_to_words(src.lower())
    # word suspected for typos
    sn, sk = src.lower(), src.lower()
    for word in words:
        if word not in vocabulary and word.isalpha():
            # top-1 accuracy
            try:
                wn, wk = fix_typo_qwerty_norvig(word), \
                         fix_typo_kgram(word, k_gram_index_github)[0]
            except:
                print(word)
            sn = sn.replace(word, wn)
            sk = sk.replace(word, wk)
    norvig += int(sn == target.lower())
    kgram += int(sk == target.lower())

print(f"Norvig accuracy ({norvig}) = {norvig / limit}")
print(f"k-gram accuracy ({kgram}) = {kgram / limit}")

만던
않
ù
ìt
ı
ı
登陆框
上右
登陆
Norvig accuracy (1258) = 0.1258
k-gram accuracy (1719) = 0.1719
